## SQL优化语句的一般步骤

1.通过show status命令了解各种SQL的执行频率

2.定位执行效率较低的SQL语句

3.通过EXPLAIN分析较低SQL的执行计划

4.通过show profile分析SQL

5.通过trace分析优化器如何选择执行计划

6.确定问题并采取相应的优化措施

参考网页：https://blog.csdn.net/ichglauben/article/details/81204375

In [2]:
import pandas as pd
import sqlalchemy
import sqlite3
pd.options.display.max_rows = 1000

In [3]:
dbname_str = {
        'oracle':"ALTER SESSION SET CURRENT_SCHEMA = \"{}\""
        , 'mysql':"USE `{}`"
        }
def __standardize_args(sql_args):
    # 检查所需参数是否都存在，规范输入的一些参数
    if not isinstance(sql_args, dict):
        raise Exception("sql_args格式错误！！！")
        
    # 规范输入的大小写
    sql_args['db_dialect'] = sql_args['db_dialect'].lower()
    sql_args['db_driver'] = sql_args['db_driver'].lower()
    
    # 不同的数据库，需要的参数不同
    if sql_args['db_dialect'] == 'oracle':
        needed_args = ['db_dialect', 'db_driver', 'host', 'user', 'password', 'sid', 'dbname']
        
        # Oracle的数据类型比较特殊
        global np_type2sql_type,sql_type2np_type,np_type2oracle_type,oracle_type2np_type
        np_type2sql_type = np_type2oracle_type
        sql_type2np_type = oracle_type2np_type
        
    elif sql_args['db_dialect'] == 'mysql':
        needed_args = ['db_dialect', 'db_driver', 'host', 'user', 'password', 'dbname']
    
    # 缺少参数则报错
    check_args = [s for s in needed_args if s not in sql_args]
    if check_args:
        raise Exception("缺少数据库参数：%s" % '，'.join(check_args))
    
    # 规定默认的参数的值 ##################################################
    if 'port' not in sql_args and sql_args['db_dialect'] == 'oracle':
        sql_args['port'] = '1521'
    if 'port' not in sql_args and sql_args['db_dialect'] == 'mysql':
        sql_args['port'] = '3306'
    if 'charset' not in sql_args:
        sql_args['charset'] = 'UTF8MB4'
        """
        这种错误很有可能是SQL驱动不完整
        也可能是数据库的编码与申请的编码不符
        1366, "Incorrect string value: '\\xD6\\xD0\\xB9\\xFA\\xB1\\xEA...' for column 'VARIABLE_VALUE' at row 484")
        """
    if 'method' not in sql_args:
        # 没有参数传入，则使用fetchall
        sql_args['method'] = None
    if 'data_type' not in sql_args:
        sql_args['data_type'] = 'list'
    #######################################################################
    return sql_args

def __sql_engine(sql_args):
    # 编辑salalchemy中的数据库参数字符串
    global eng_str
    db_dialect = sql_args['db_dialect']
    engine = sqlalchemy.create_engine(eng_str[db_dialect].format(**sql_args))#, echo=True)
    return engine

eng_str = {
        'oracle':"{db_dialect}+{db_driver}://{user}:{password}@{host}:{port}/{sid}?charset={charset}"
        , 'mysql': "{db_dialect}+{db_driver}://{user}:{password}@{host}:{port}/{dbname}?charset={charset}"
        }
sql_args = {
    'db_dialect': 'MySQL'
    , 'db_driver': 'pymysql'
    , "host": "192.168.50.190"
    , "user": "lyb"
    , "password": "122321"
    , 'dbname': 'qy_data'
    , 'data_type': 'DataFrame'
}
sql_args = __standardize_args(sql_args)
mysql_engine = __sql_engine(sql_args)
sqlite_engine = sqlalchemy.create_engine('sqlite:///param_annotation.db')

## 通过show status命令了解各种SQL的执行频率

In [ ]:
with sqlite_engine.connect() as conn:
    resp = conn.execute('SELECT Variable_name, Annotation FROM mysql_status')
    annotation = dict(resp.fetchall())

sql_str = 'show status'
res = pd.read_sql(sql_str, mysql_engine)
res['Annotation'] = res['Variable_name'].apply(lambda s: annotation.get(s))
print("显示了当前session中所有统计参数的值（前10）")
print(res[res['Variable_name'].str[0:4]=='Com_'].head(10))

print("\n前者中比较关注的统计参数")
important_var = ("Com_select","Com_insert","Com_update","Com_delete")
print(res[res['Variable_name'].isin(important_var)])

print("\nInnodb中比较关注的统计参数")
"""
文中说
以上参数对所有存储引擎都会进行累计
下面参数只对InnoDB进行累计

结果前者基本都是0，后者却有统计数据
"""
innodb_var = ("Innodb_rows_read","Innodb_rows_inserted","Innodb_rows_updated","Innodb_rows_deleted")
print(res[res['Variable_name'].isin(innodb_var)])

print("\n事务型应用")
transaction_var = ("Com_commit","Com_rollback")
print(res[res['Variable_name'].isin(transaction_var)])

## 定位执行效率较低的SQL语句

参考文章
https://blog.csdn.net/TyphoonHao/article/details/91562084
https://blog.csdn.net/qq_24025219/article/details/105137540

## 通过EXPLAIN分析较低SQL的执行计划

| field1        | field2          | meaning                                                      |
| ------------- | --------------- | ------------------------------------------------------------ |
| select_type   | &nbsp;           | 表示SELECT的类型                                             |
| &nbsp;         | SIMPLE          | 简单表，不使用表连接或者子查询                               |
| &nbsp;         | PRIMARY         | 主查询，外层的查询                                           |
| &nbsp;         | UNION           | UNION中的第二个或者后面的查询语句                            |
| &nbsp;         | SUBQUERY        | 子查询中的第一个select                                       |
| table         | &nbsp;           | 输出结果集的表                                               |
| type          | &nbsp;           | 表示MySQL在表中找到所需行的方式，或者访问类型                |
| &nbsp;         | all             | 全表扫描，MySQL遍历全表来找到匹配的行                        |
| &nbsp;         | index           | 索引全扫描，Mysql遍历整个索引查询匹配行                      |
| &nbsp;         | range           | 索引范围扫描，常见于 < <= > >= between                       |
| &nbsp;         | ref             | 使用非唯一索引扫描或唯一索引的前缀扫描，返回某个单独值得记录行，ref还经常出现在join操作中 |
| &nbsp;         | eq_ref          | 类似ref，区别使用的是唯一索引 对于每个索引键值。表中只有一个值（就是多表连接中使用primary key或者unique index作为关联条件） |
| &nbsp;         | const,system    | 单表中最多有一个匹配行，查询起来很迅速。这个匹配行中的其他列值可以被optimizer在当前查询中当作常量处理 |
| &nbsp;         | NULL            | 不使用访问表或者索引，直接得到结果                           |
| &nbsp;         | ref_or_null     | 类似ref，区别条件中包含对NULL查询                            |
| &nbsp;         | index_merge     | 索引合并优化                                                 |
| &nbsp;         | unique_subquery | in的后面是一个查询主键字段的子查询                           |
| &nbsp;         | index_subquery  | 与unique_subquery类似，区别在in后面是查询非唯一索引字段的子查询等 |
| possible_keys | &nbsp;           | 表示查询时可能使用的索引                                     |
| key           | &nbsp;           | 表示实际使用的索引                                           |
| key_len       | &nbsp;           | 使用到索引字段的长度                                         |
| rows          | &nbsp;           | 扫描行数量                                                   |
| EXTRA         | &nbsp;           | 执行情况的说明和描述，包含不适合在其他列中显示但对执行计划非常重要的额外信息 
